In [1]:
import pandas as pd
import numpy as np

In [2]:
review_df = pd.read_csv("/content/drive/MyDrive/ml-data/Sentiment-analysis/labeledTrainData.tsv", header=0, sep="\t", quoting=3)
review_df.head(4)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."


####Supervised way

In [3]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [4]:
review_df['review'] = review_df['review'].replace('<br />', ' ')

In [5]:
import re

review_df['review'] = review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]", " ", x))

In [6]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df['review']
print(class_df.shape, feature_df.shape)
train_feature_df, test_feature_df, train_class_df, test_class_df = train_test_split(feature_df, class_df, test_size=.2, random_state=156)

(25000,) (25000,)


In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [8]:
train_class_df.shape, train_feature_df.shape

((20000,), (20000,))

In [9]:
pipeline = Pipeline([
                     ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
                     ('lr_clf', LogisticRegression(C=10))
])

pipeline.fit(train_feature_df, train_class_df)
pred = pipeline.predict(test_feature_df)
pred_probs = pipeline.predict_proba(test_feature_df)
print(accuracy_score(test_class_df, pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8932


In [10]:
print(roc_auc_score(test_class_df, pred_probs[:,1]))

0.9556523757684425


In [11]:
pipeline = Pipeline([
                     ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
                     ('lr_clf', LogisticRegression(C=10)),
])

pipeline.fit(train_feature_df, train_class_df)
pred = pipeline.predict(test_feature_df)
pred_probs = pipeline.predict_proba(test_feature_df)

print(accuracy_score(test_class_df, pred))
print(roc_auc_score(test_class_df, pred_probs[:,1]))

0.9006
0.9649076267930328


####Unsupervised way

In [12]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [13]:
from nltk.corpus import wordnet as wn
word = "present"
synsets = wn.synsets(word)
print(type(synsets))
print(len(synsets))
print(synsets)

<class 'list'>
18
[Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [14]:
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets(word))
print(type(senti_synsets))
print(len(senti_synsets))
print(senti_synsets)

<class 'list'>
18
[SentiSynset('present.n.01'), SentiSynset('present.n.02'), SentiSynset('present.n.03'), SentiSynset('show.v.01'), SentiSynset('present.v.02'), SentiSynset('stage.v.01'), SentiSynset('present.v.04'), SentiSynset('present.v.05'), SentiSynset('award.v.01'), SentiSynset('give.v.08'), SentiSynset('deliver.v.01'), SentiSynset('introduce.v.01'), SentiSynset('portray.v.04'), SentiSynset('confront.v.03'), SentiSynset('present.v.12'), SentiSynset('salute.v.06'), SentiSynset('present.a.01'), SentiSynset('present.a.02')]


In [15]:
father = swn.senti_synset("father.n.01")
fabulous = swn.senti_synset("fabulous.a.01")

print(father.pos_score())
print(father.neg_score())
print(father.obj_score())

print(fabulous.pos_score())
print(fabulous.neg_score())
print(fabulous.obj_score())

very = swn.senti_synset("very.a.01")
print(very.pos_score())
print(very.neg_score())
print(very.obj_score())

0.0
0.0
1.0
0.875
0.125
0.0
0.5
0.0
0.5


In [17]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def vader_polarity(review, threshold=0.1):
  analyzer = SentimentIntensityAnalyzer()
  scores = analyzer.polarity_scores(review)
  return 1 if scores['compound'] > threshold else 0  

In [18]:
from sklearn.metrics import confusion_matrix

review_df['vader_pred'] = review_df['review'].apply(lambda x : vader_polarity(x, 0.1))
y_target = review_df['sentiment'].values
vader_pred = review_df['vader_pred']

print(confusion_matrix(y_target, vader_pred))
print(np.round(accuracy_score(y_target, vader_pred), 2))

[[ 6736  5764]
 [ 1866 10634]]
0.69


In [23]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x', 'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med']

news_df = fetch_20newsgroups(subset="all", remove=('headers', 'footers', 'quotes'), categories=cats, random_state=0)
count_vect = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english', ngram_range=(1,2))

In [72]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(news_df.data, test_size=0.1)

In [25]:
feature_vect = count_vect.fit_transform(news_df.data)
print(feature_vect.shape)

(7862, 1000)


In [26]:
lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(feature_vect)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [27]:
print(lda.components_.shape)
print(lda.components_)

(8, 1000)
[[3.60992018e+01 1.35626798e+02 2.15751867e+01 ... 3.02911688e+01
  8.66830093e+01 6.79285199e+01]
 [1.25199920e-01 1.44401815e+01 1.25045596e-01 ... 1.81506995e+02
  1.25097844e-01 9.39593286e+01]
 [3.34762663e+02 1.25176265e-01 1.46743299e+02 ... 1.25105772e-01
  3.63689741e+01 1.25025218e-01]
 ...
 [3.60204965e+01 2.08640688e+01 4.29606813e+00 ... 1.45056650e+01
  8.33854413e+00 1.55690009e+01]
 [1.25128711e-01 1.25247756e-01 1.25005143e-01 ... 9.17278769e+01
  1.25177668e-01 3.74575887e+01]
 [5.49258690e+01 4.47009532e+00 9.88524814e+00 ... 4.87048440e+01
  1.25034678e-01 1.25074632e-01]]


In [52]:
def display_topics(model, feature_names, no_top_words):
  for i, aline in enumerate(model.components_):
    print("======================= Topic : {0} =====================".format(i+1))
    sorted_line_index = (aline.argsort()[::-1])[:no_top_words]
    ranked_features = [feature_names[j] for j in sorted_line_index]
    print("\t".join(ranked_features))

In [53]:
display_topics(lda, count_vect.get_feature_names(), 10)

======================= Topic : 1 =====================
year	10	game	medical	health	team	12	20	disease	cancer
======================= Topic : 2 =====================
don	just	like	know	people	said	think	time	ve	didn
======================= Topic : 3 =====================
image	file	jpeg	program	gif	images	output	format	files	color
======================= Topic : 4 =====================
like	know	don	think	use	does	just	good	time	book
======================= Topic : 5 =====================
armenian	israel	armenians	jews	turkish	people	israeli	jewish	government	war
======================= Topic : 6 =====================
edu	com	available	graphics	ftp	data	pub	motif	mail	widget
======================= Topic : 7 =====================
god	people	jesus	church	believe	christ	does	christian	say	think
======================= Topic : 8 =====================
use	dos	thanks	windows	using	window	does	display	help	like


In [75]:
cv = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english', ngram_range=(1,2))
train_cnt = cv.fit_transform(train_data)
test_cnt = cv.transform(test_data)

lda = LatentDirichletAllocation(n_components=8)
lda.fit(train_cnt)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [76]:
display_topics(lda, cv.get_feature_names(), 15)

======================= Topic : 1 =====================
know	like	does	thanks	use	just	don	help	ve	problem	need	want	work	used	using
======================= Topic : 2 =====================
year	good	don	time	think	years	like	game	just	medical	better	use	10	long	disease
======================= Topic : 3 =====================
armenian	israel	jews	turkish	people	armenians	government	israeli	jewish	war	turkey	arab	new	000	genocide
======================= Topic : 4 =====================
file	server	use	window	output	program	widget	motif	sun	set	display	x11	entry	include	mit
======================= Topic : 5 =====================
just	don	said	know	people	like	didn	time	bike	say	going	went	right	ll	did
======================= Topic : 6 =====================
image	jpeg	file	color	images	files	gif	format	bit	use	data	display	program	software	version
======================= Topic : 7 =====================
edu	com	dos	graphics	mail	software	dos dos	available	windows	information	ftp	computer	send

In [84]:
lda.transform(test_cnt).shape

(787, 8)